# Synacor Challenge VM

**Solution Started**: July 7, 2025  
**Author**: Abbas Moosajee

This notebook loads and runs the Synacor challenge binary using a custom `VirtualMachine` implementation, highlights challenge codes, and displays the interactive game output in styled HTML.

In [ ]:
import os, re, time, copy, html, hashlib
from collections import defaultdict, deque
from IPython.display import display, HTML
from VirtualMachine import VirtualMachine
start_time = time.time()

In [ ]:
# Load the binary program file
program_file = "challenge.bin"
file_path = os.path.join(os.getcwd(), program_file)  # assumes file in current working dir
vm_program = open(file_path, "rb").read()

In [ ]:
class SynacorConsole:
    track_backwards = {
        # "north":"south", "south":"north", "east":"west","west":"east",
        # "up":"down", "down": "up", "forward":"back", "back":"forward"
        }

    def __init__(self, software: bytes, spec_code: str, visualize: bool = True):
        self.software = software
        self.visualize = visualize
        self.console = self.reset_console()
        self.__extract_codes(" ", [spec_code])

    def reset_console(self):
        console = VirtualMachine(self.software)
        self.game_map = defaultdict(list)
        self.overall_commands = []
        self.challenge_codes = {}
        self.valid_codes = set()
        self.code_no = 0
        return console

    @staticmethod
    def md5_hash(code: str) -> str:
        return hashlib.md5(code.encode('utf-8')).hexdigest()

    def __extract_codes(self, text, codes = []):
        """Extract mixed-case challenge codes and store them with their MD5 hashes."""
        codes_found = re.findall(r"[A-Za-z]*[a-z]+[A-Z]+[a-z]+[A-Za-z]*", text)
        codes.extend(codes_found)

        new_codes = []
        for code in codes:
            if code not in self.valid_codes:
                self.code_no += 1
                self.challenge_codes[self.code_no] = (code, self.md5_hash(code))
                new_codes.append(code)
                self.valid_codes.add(code)
        return new_codes

    def __format_terminal(self, text, code_color, input_color):
        """Highlight any detected code-like strings in the terminal."""
        lines = []
        for line in text.splitlines():
            # Simple input highlight: lines starting with '>'
            if line.strip().startswith('>>'):
                line = f"<span style='color:{input_color}'>{line}</span>"
            else:
                line = re.sub(      # Example: highlight code-like patterns
                    r"\b((?:[a-z]+[A-Z]+[a-z]|[A-Z]+[a-z]+[A-Z])\w*)\b",
                    rf"<span style='color:{code_color}'>\1</span>", line
                )
            lines.append(line)
        return "\n".join(lines)

    def display_terminal(self, terminal_text, actions):
        """Format and display terminal output with highlighted codes and interleaved actions."""
        self.color_dict = {
            "background": "#282828", "terminal": "#33FF00",
            "codes": "#FF0000", "input": "#0073FF"
        }
        html_valid_text = html.escape(terminal_text)
        # Normalize line endings and remove excessive empty lines
        html_valid_text = re.sub(r'\n{3,}', '\n\n', html_valid_text.strip())  # At most 2 newlines
        parts = re.split(r'(What do you do\?)', html_valid_text)

        output_lines = []

        if actions:
            output_lines.append(f">> {actions[0]}")
        action_index = 1 if actions else 0

        # Now interleave remaining actions after each prompt
        i = 0
        while i < len(parts):
            output_lines.append(parts[i])
            input_prompt = "What do you do?"
            if i + 1 < len(parts) and parts[i + 1] == input_prompt:
                output_lines.append(input_prompt) # Append prompt
                if action_index < len(actions):   # Append next action if available
                    output_lines.append(f">> {actions[action_index]}")
                    action_index += 1
                i += 2
            else:
                i += 1

        combined_text = "\n".join(output_lines)
        longest_line = max(max(len(line) for line in combined_text.split('\n')), 150)

        html_formatted = (
            f"<div style='background-color: {self.color_dict['background']}; "
            f"width: {longest_line + 2}ch; padding: 0.75ex;'>"
            f"<pre style='background-color: {self.color_dict['background']}; "
            f"color: {self.color_dict['terminal']}; margin: 0; font-family: monospace; "
            f"width: {longest_line + 2}ch;'>"
            + self.__format_terminal(combined_text, self.color_dict['codes'], self.color_dict['input'])
            + "</pre></div>"
        )

        display(HTML(html_formatted))

    def play_game_manually(self, actions = []):
        """
        Game Instructions:
        - `look`: You may merely 'look' to examine the room, or you may 'look ' (such as 'look chair') to examine something specific.
        - `go`: You may 'go ' to travel in that direction (such as 'go west'), or you may merely '' (such as 'west').
        - `inv`: To see the contents of your inventory, merely 'inv'.
        - `take`: You may 'take ' (such as 'take large rock').
        - `drop`: To drop something in your inventory, you may 'drop '.
        - `use`: You may activate or otherwise apply an item with 'use '.
        """
        if actions:
            self.overall_commands.extend(actions)
        full_terminal = self.console.run_computer(actions)
        current_terminal = full_terminal[-1]
        self.__extract_codes(current_terminal)
        if self.visualize:
            self.display_terminal(current_terminal, actions)
        return self.overall_commands

    def __parse_game_state(self, terminal):
        lines = terminal.splitlines()
        location, description, things, exits = ("", "", [], [])

        current_section = None
        for line_no, line in enumerate(lines):
            line = line.strip()
            if line.startswith("==") and line.endswith("=="):
                location = line.strip("= ").strip()
                description = lines[line_no + 1]
            elif line.startswith("Things of interest here:"):
                current_section = "things"
            elif line.startswith("There") and "exit" in line:
                current_section = "exits"
            elif line.startswith("-") and current_section:
                item = line[2:].strip()
                if current_section == "things":
                    things.append(item)
                elif current_section == "exits":
                    exits.append(item)
            else:
                current_section = None

        return location, description, things, exits

    def create_room_id(self, room_description):
        h = self.md5_hash(room_description.strip().lower())
        # Take first 4 hex digits, convert to int, and mod 10000 to get a 4-digit number
        return int(h[:8], 16) % 10000

    def __use_items(self, item):
        """Return actions to collect and use the item, and what items will be added to inventory."""
        if item == 'empty lantern':
            return [f"take {item}", 'use can', "use lantern"], {"lantern"}
        elif "coin" in item:
            return [f"take {item}"], {item}
        else:
            return [f"take {item}", f"use {item}"], {item}

    def __solve_coins_puzzle(self, inventory, game_copy):
        look_coins = [(f"look {item}") for item in inventory if 'coin' in item]
        *_, final_terminal = game_copy.run_computer(look_coins)
        self.display_terminal(final_terminal, look_coins)
        # print(final_terminal)
        return

    def bfs_exploration(self):
        """Perform BFS to explore and map out the game world."""
        collect_coins = {'concave coin', 'shiny coin', 'red coin', 'blue coin', 'corroded coin'}
        visited, bfs_history = (set(), defaultdict(tuple))
        steps, MAX_STEPS = (0, 750)
        coins_puzzle = False

        # Each entry: (console_state, pending_actions, path_so_far, action_history)
        # Use deque for efficient popping from the left
        queue = deque([(self.console, [], [], set())])

        while queue and steps < MAX_STEPS:
            state, pending_actions, action_history, game_inv = queue.popleft()
            current_actions = []
            steps += 1

            # Run actions on a fresh copy of the game state
            game_copy = state.replicate(steps)
            *_, final_terminal = game_copy.run_computer(pending_actions)
            bfs_history[steps] = (game_copy, action_history)
            self.__extract_codes(final_terminal)

            # Parse game state
            room, desc, all_items, room_exits = self.__parse_game_state(final_terminal)
            room_id = self.create_room_id(desc)

            # Skip if already visited with current inventory
            if (room_id, tuple(game_inv)) in visited:
                continue
            visited.add((room_id, tuple(game_inv)))

            if (
                len(collect_coins & game_inv) == 5 and
                'You stand in the massive central hall of these ruins.' in desc
                ):
                coin_solution = self.__solve_coins_puzzle(game_inv, game_copy)
            #     print(action_history)
            #     self.__solve_coins_puzzle(action_history, game_copy)
            #     full_terminal = '\n'.join(game_copy.output_terminal[1:])
            #     self.display_terminal(full_terminal, action_history)

            # Clone game_inv for branching paths
            base_inv = game_inv.copy()

            # Handle item collection
            for item in all_items:
                # Skip collecting lantern unless can is collected
                if (item == "empty lantern") and ("can" not in game_inv):
                    continue
                item_actions, collected = self.__use_items(item)

                base_inv.update(collected)
                current_actions.extend(item_actions)
                action_history.extend(item_actions)
                queue.append((game_copy, item_actions, action_history, base_inv))

            # Explore exits
            for direction in room_exits:
                is_dark_passage = (desc == "You are in a dark, narrow passage.")

                # If override: move twice in the same direction
                dir_sequence = [direction, direction] if is_dark_passage else [direction]

                next_actions = current_actions + dir_sequence
                next_history = action_history + dir_sequence
                queue.append((game_copy, next_actions, next_history, base_inv))

        print("steps:",steps)
        return action_history

    def play_game(self, actions = []):
        return self.bfs_exploration()


In [ ]:
console = SynacorConsole(vm_program, "LDOb7UGhTi")
# command_list = console.play_game()
command_list = console.bfs_exploration()

In [ ]:
# manual_console = SynacorConsole(vm_program, "LDOb7UGhTi",)
# manual_console.play_game_manually()
# game_actions = [
#     'take tablet', 'use tablet', 'doorway', 'north', 'north', 'bridge', 'continue', 'down', 'west', 'passage', 'ladder',
#     'west', 'south', 'north', 'take can', 'use can', 'west', 'ladder', 'cavern', 'east', 'east', 'take empty lantern',
#     'use can', 'use lantern', 'west', 'west', 'passage', 'darkness', 'continue', 'west', 'west', 'west', 'west', 'north',
#     'take red coin', 'north', 'east', 'take concave coin', 'down', 'take corroded coin', 'up', 'west', 'west',
#     'take blue coin', 'up', 'take shiny coin', 'down', 'east', 'inv',
#     "use blue coin", "use red coin", "use shiny coin", "use concave coin", "use corroded coin", "north", "take teleporter", "use teleporter"
#     ]
# all_actions = manual_console.play_game_manually(game_actions)
# # print(manual_console.challenge_codes)

In [ ]:
for code_no, code_group in console.challenge_codes.items():
    print(f"Code {code_no}: {code_group[0]} -> {code_group[1]}")
print(f"Execution Time = {time.time() - start_time:.5f}s")